In [14]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [56]:
data_f = pd.read_csv("C:\RUTU\WF_project\WF-CMT-6-MBS\Combined Dataset\Dataset.csv")
print(data_f.head())


   Loan Identifier  Monthly Reporting Period  Channel  Original Interest Rate  \
0     100007365142                     12000        2                     8.0   
1     100007365142                     22000        2                     8.0   
2     100007365142                     32000        2                     8.0   
3     100007365142                     42000        2                     8.0   
4     100007365142                     52000        2                     8.0   

   Current Interest Rate  Original UPB  Current Actual UPB  \
0                    8.0       75000.0             75000.0   
1                    8.0       75000.0             75000.0   
2                    8.0       75000.0             75000.0   
3                    8.0       75000.0             75000.0   
4                    8.0       75000.0             75000.0   

   Original Loan Term  Origination Date  First Payment Date  ...  \
0                 360            121999               22000  ...   
1   

In [16]:
cols=data_f.columns
print(len(cols))
print(cols)

37
Index(['Loan Identifier', 'Monthly Reporting Period', 'Channel',
       'Original Interest Rate', 'Current Interest Rate', 'Original UPB',
       'Current Actual UPB', 'Original Loan Term', 'Origination Date',
       'First Payment Date', 'Remaining Months To Maturity',
       'Original Loan to Value Ratio (LTV)', 'Debt-To-Income (DTI)',
       'Borrower Credit Score at Origination',
       'First Time Home Buyer Indicator', 'Loan Purpose', 'Property Type',
       'Number of Units', 'Occupancy Status',
       'Metropolitan Statistical Area (MSA)', 'Zip Code Short',
       'Interest Only Loan Indicator', 'Current Loan Delinquency Status',
       'Modification Flag', 'Zero Balance Code', 'Special Eligibility Program',
       'Property Valuation Method', 'High Balance Loan Indicator',
       'Unemployment_rate', 'Divorce_rate', 'Inflation_rate', 'EMI',
       'UPB_diff', 'Interest_Component', 'PREPAYMENT_AMT', 'Previous_UPB',
       'Remaining MFM'],
      dtype='object')


In [57]:
time_steps = 6

In [58]:
from numpy import array

def split_sequences(dataf, n_steps): 
    sequences = dataf.copy()
    #print(sequences.columns)   
    y_df = sequences['Current Actual UPB']
    x_df = sequences.drop(['Loan Identifier','Current Actual UPB','UPB_diff','EMI','Interest_Component','PREPAYMENT_AMT','Remaining Months To Maturity'], axis = 1)
    
    X,Y = list(), list()
    print(len(x_df.columns))
    x_arr = np.asarray(x_df.values.tolist())
    y_arr = np.asarray(y_df.values.tolist()).reshape(len(y_df.values.tolist()),1)
    print(x_arr.shape)
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + time_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern

        #print(len(x_arr))
        seq_x = x_arr[i:end_ix, :]
        # print(seq_x.shape)
        seq_y = y_arr[end_ix-1,-1]
        X.append(seq_x)
        Y.append(seq_y)
        
        
   
    return array(X), array(Y)

In [59]:
X,Y= split_sequences(data_f, time_steps)

30
(1256468, 30)


In [60]:
X.shape

(1256462, 6, 30)

In [35]:
Y.shape

(1256462,)

In [87]:
# time_steps = 6


In [68]:

X_train, X_test, Y_train, Y_test = train_test_split(X,Y ,train_size=0.8, test_size = 0.2, shuffle=False)

In [37]:
X_train.shape

(1005169, 6, 29)

In [52]:

# X_train, X_test, Y_train, Y_test = train_test_split(X.squeeze(0),Y.squeeze(0) ,random_state = 0,train_size=0.8, test_size = 0.2, shuffle=False)
# X_train = np.expand_dims(X_train,axis=0)
# X_test = np.expand_dims(X_test,axis=0)
# Y_train = np.expand_dims(Y_train,axis=0)
# Y_test = np.expand_dims(Y_test,axis=0)
# x=X_train[:,:,0:3]
# print(X_train.shape)

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [69]:
n_features = X_train.shape[2]
print(n_features)

30


In [70]:
n_features

30

In [33]:
X_train.shape

(1005169, 6, 28)

In [34]:
hiddensizes = [15,20, 30, 40]
dropouts = [0,0.8,0.15,0.2]
batch_sizes = [32,64,128]
optimizers = ['rmsprop', 'adam']
for dropout in dropouts:
    for batch_size in batch_sizes:
        for optimizer in optimizers:
            for hidden in hiddensizes:
                print('dropout:' + str(dropout) + '\t' + 'batchsize:' + str(batch_size) + '\t' + 'optimzer:' + str(optimizer)  + '\t' + 'Hidden_U:' + str(hidden))

                lstm_model = Sequential()
                #hiddensize1 = X.shape[2]

                lstm_model.add(LSTM(units = hidden, dropout=dropout, return_sequences = True, input_shape = (time_steps, n_features)))
                lstm_model.add(LSTM(units = hidden, dropout=dropout, return_sequences = True))
                #lstm_model.add(LSTM(units = hiddensize1, dropout=0.2, return_sequences = True))

                lstm_model.add(Dense(n_features))
                #weights = np.array([0.005,0.24,0.75,0.005])
                #weights = np.ones((4,))*1.0
                lstm_model.compile(optimizer = optimizer, loss = 'mse', metrics = ['categorical_accuracy'])
                lstm_model.fit(X_train, Y_train,batch_size=batch_size, epochs=4, verbose=1,validation_data=(X_test, Y_test*1.0))

dropout:0	batchsize:32	optimzer:rmsprop	Hidden_U:15
Epoch 1/4


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/mean_squared_error/BroadcastGradientArgs' defined at (most recent call last):
    File "c:\Users\vishw\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\vishw\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\vishw\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\vishw\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\vishw\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\vishw\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 195

In [63]:
def check_accuracy(Y_test,y_pred):
    from sklearn.metrics import r2_score,accuracy_score
    print(r2_score(y_pred,Y_test))
    #print(metrics.confusion_matrixY, predictTrain)

In [50]:
hiddensizes = [15,20, 30, 40]
for hidden in hiddensizes:
    print('Hidden size'+str(hidden))
    model = Sequential()
    model.add(LSTM(hidden, activation='relu', return_sequences=True, input_shape=(time_steps,n_features)))
    model.add(LSTM(hidden, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse',metrics = ['categorical_accuracy'])
    # fit model
    model.fit(X_train, Y_train, epochs=5, verbose=0)
    y_pred=model.predict(X_test)
    check_accuracy(Y_test,y_pred)
    

Hidden size15


KeyboardInterrupt: 

In [71]:
hidden=64
print('Hidden size '+str(hidden))
model = Sequential()
model.add(LSTM(hidden, activation='relu', return_sequences=True, input_shape=(time_steps,n_features)))
# model.add(Dropout(0.2))
model.add(LSTM(hidden, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse',metrics = ['acc'])
    # fit model
model.fit(X_train, Y_train,epochs=5, verbose=1)
y_pred=model.predict(X_test)
check_accuracy(Y_test,y_pred)
    

Hidden size 64
Epoch 1/5
31412/31412 [==============================] - 228s 7ms/step - loss: 858649472.0000 - acc: 1.9997e-04
Epoch 2/5
31412/31412 [==============================] - 226s 7ms/step - loss: 578826560.0000 - acc: 5.2727e-04
Epoch 3/5
31412/31412 [==============================] - 1170s 37ms/step - loss: 333699712.0000 - acc: 0.0033
Epoch 4/5
31412/31412 [==============================] - 224s 7ms/step - loss: 142135888.0000 - acc: 0.0071
Epoch 5/5
7853/7853 [==============================] - 23s 3ms/step
0.9258918647005214


In [72]:
Y_test

array([214581.22, 214226.92, 213871.41, ..., 499853.61, 498960.1 ,
       498064.31])

In [73]:
y_pred

array([[215186.61],
       [214530.12],
       [213880.36],
       ...,
       [498668.2 ],
       [501442.84],
       [498412.12]], dtype=float32)

In [74]:
import pickle
with open('LSTM_model_64_ep5_t6','wb') as model_lstm:
    pickle.dump(model,model_lstm) 